In [1]:
import pandas as pd
from numpy import mean, std
import datetime
from matplotlib.pyplot import *

from functions.rationing import rationing_monteCarlo
# Adecuando los datos de entrada:

## importando datos de irradiancia
df_irradiancia = pd.read_excel('data.xlsx', sheet_name='NSRDB', usecols=['Month','Day','Hour','Minute','Clearsky GHI','GHI'] )
## reemplanzando headers con espacios por barras
df_irradiancia.columns = [c.replace(' ', '_') for c in df_irradiancia.columns]
df_irradiancia['K_t'] = ( df_irradiancia['GHI'] / df_irradiancia['Clearsky_GHI'] )

## importando datos del perfil de consumo horario en un dia
consumo_horario = pd.read_excel('data.xlsx', sheet_name='Edt',index_col='hora')
## generando una lista con los valores de energia en intervalos de una hora
energia_hora = [ i.energia for i in consumo_horario.itertuples() ]


In [2]:
# Cantidad de módulos y baterías
n_modulos = 8
n_baterias = 4

# Precio de cada módulo
p_modulo = 798011.78
# Precio de cada batería
p_bateria = 1787409

# Función simplificada de costos
costo = n_modulos * p_modulo + n_baterias * p_bateria

r_mc = rationing_monteCarlo(    iterations=100,                     # numero de iteraciones [und]
                                irradiance_data=df_irradiancia,     # datos de irradiancia [DataFrame de Pandas]
                                time_zone=-5,                       # horas [H-GMT]
                                latitude=4.591480,                  # latitud [° Decimales (DD)]
                                longitude=-74.691412,               # longitud [° Decimales (DD)]
                                altitude=321,                       # altitud [m]
                                climate_type='Tropical',            # tipo de clima ['Tropical', 'Midlatitude summer', 'Subarctic summer', 'Midlatitude winter']
                                year=2022,                          # año[AAAA]
                                surf_azm=180,                       # angulo azimutal del arreglo de módulos [°]
                                surf_tilt=4,                        # angulo de inclinación del arreglo de módulos [°]
                                albedo=0.17,                        # albedo [adimensional]
                                energy_per_hour=energia_hora,       # datos horarios del consumo energético cada hora en un dia normal [Wh]
                                n_m=n_modulos,                      # numero de módulos en el arreglo [und]
                                A_m=2.5,                            # area efectiva del modulo [m^2]
                                eta_m=19,                           # eficiencia del modulo [%]
                                n_b=n_baterias,                     # numero de baterías [und]
                                v_b=12,                             # tension nominal de la batería [Vdc]
                                E_b_max=300,                        # capacidad de la batería [Ah]
                                l_b_min=40,                         # nivel mas bajo de descarga de la batería [%] (100% - profundidad de descarga)
                                l_b_0=50                            # nivel de carga inicial del sistema de almacenamiento [%]
                            )

print('costo: $', costo )
print('máximo racionamiento:', round( max(r_mc) * 100 , 2 ), '%' )
print('media:',round(  mean(r_mc) * 100, 2 ), '%' )
print('desviación estándar:', round( std(r_mc) * 100, 2 ), '%' )


costo: 13533730.24
máximo racionamiento: 4.11 %
media: 2.56 %
desviación estándar: 0.49 %
